Connect to sheets

In [1]:
import pandas as pd
import streamlit as st
from streamlit_gsheets import GSheetsConnection
from datetime import datetime

In [2]:
url = "https://docs.google.com/spreadsheets/d/1xVKoVmKZOpNZ1oBySdLTE5GXZwN66UNoKjaBoEifZss/edit?usp=sharing"

# Create a connection object
conn = st.connection("gsheets", type=GSheetsConnection)

# Create raw Budget dataframe
budget_df = conn.read(spreadsheet =url, usecols=list(range(15)), worksheet=832591380, ttl=5)

# Create raw Actual dataframe
actual_df = conn.read(spreadsheet =url, usecols=list(range(15)), worksheet=487806377, ttl=5)

2025-05-06 06:53:32.803 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-06 06:53:34.391 
  command:

    streamlit run /Users/oscaraguilarembila/miniconda3/envs/ML/lib/python3.13/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-05-06 06:53:34.395 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-06 06:53:34.398 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-06 06:53:34.402 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-06 06:53:34.404 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-06 06:53:34.495 No runtime found, using MemoryCacheStorageManager
2025-05-06 06:53:34.508 No runtime found, using MemoryCacheStorageMa

In [ ]:
# Make a copy of both dfs
b_df = budget_df.copy()
a_df = actual_df.copy()

# Rename columns
b_df.columns = ["Type", "Buckets"] + list(budget_df.iloc[3])[2:] # Budget
a_df.columns = ["Type", "Buckets"] + list(actual_df.iloc[3])[2:] # Actual

# Remove unnecessary rows
b_df = b_df.iloc[26:48] # Budget
a_df = a_df.iloc[26:48] # Actual

# Store Type and Buckets array in a Dataframe
categories = pd.DataFrame({
	"Type": b_df.Type,
	"Bucket": b_df.Buckets
})

# Remove unnecessary columns
b_df.drop(b_df.columns[[0, -1]], axis=1, inplace=True) # Budget
a_df.drop(a_df.columns[[0, -1]], axis=1, inplace=True) # Actual

# Reset index for all dataframes
b_df = b_df.reset_index(drop=True).drop("Buckets", axis=1) # Budget
a_df = a_df.reset_index(drop=True).drop("Buckets", axis=1) # Actual
categories = categories.reset_index(drop=True)

In [6]:
b_df

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dic
0,729,576,545.18,608.11,400,400,400,400,400,400,400,400
1,36,106,73.91,0.00,50,50,50,50,50,50,50,50
2,0,30,376.20,10.00,30,30,30,30,30,30,30,30
3,68,0,0.00,39.99,20,20,20,20,20,20,20,20
4,0,0,0.00,62.00,15,15,15,15,15,15,15,15
5,0,0,0.00,0.00,15,15,15,15,15,15,15,15
6,990,0,990.00,726.66,500,500,500,500,500,500,500,500
7,12,30,9.75,509.00,20,20,20,20,20,20,20,20
8,30,60,30.00,32.00,30,30,30,30,30,30,30,30
9,0,0,19.46,0.00,10,10,10,10,10,10,10,10


In [ ]:
# Remove comas from all columns
budget = b_df.applymap(lambda x: str(x).replace(',', ''))
for col in a_df.select_dtypes(include='object'):
    a_df[col] = a_df[col].str.replace(',', '', regex=False)

# Convert to numbers
budget = budget.apply(pd.to_numeric)
actuals = a_df.apply(pd.to_numeric)

# Fill NA with 0
budget = budget.fillna(0)
actuals = actuals.fillna(0)

# Get today's month
td = datetime.today().strftime("%b")

/var/folders/g8/7typdv_j24q8mrqfwd3816r00000gn/T/ipykernel_775/3120697006.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  budget = b_df.applymap(lambda x: str(x).replace(',', ''))


In [14]:
categories.shape

(22, 2)

In [15]:
categories.Bucket

0                  Food
1        House Supplies
2           Health Care
3             Self Care
4     Carro Maintenance
5                  Ropa
6               Escuela
7                 Isaac
8                  Adri
9                 Eliam
10            Dates/Fun
11        Subscriptions
12                Gifts
13            Fast Food
14            Junk Food
15              Viajes 
16       Emergency Fond
17            Debt/Fees
18              Savings
19            Investing
20              Negocio
21         Miscelaneous
Name: Bucket, dtype: object

In [20]:
final_df = pd.DataFrame({
    'Type': categories.Type,
    'Category': categories.Bucket,
    'Balance': budget[td] - actuals[td],
    'Budget': budget[td]
})
final_df

,Type,Category,Balance,Budget
0,N,Food,225.0,400
1,N,House Supplies,50.0,50
2,N,Health Care,30.0,30
3,N,Self Care,20.0,20
4,N,Carro Maintenance,15.0,15
5,N,Ropa,15.0,15
6,W,Escuela,500.0,500
7,W,Isaac,20.0,20
8,W,Adri,30.0,30
9,W,Eliam,10.0,10


In [ ]:
# Save to pickle
final_df.to_pickle("April")

In [21]:
# Group by Type
final_df.groupby("Type").agg({'Balance':'sum', 'Budget':'sum'}).reset_index()

,Type,Balance,Budget
0,N,355.0,530
1,SD,1022.0,1300
2,W,785.0,785


In [22]:
# Filter Categories of interest
final_df[final_df['Balance'] > 0]

,Type,Category,Balance,Budget
0,N,Food,225.0,400
1,N,House Supplies,50.0,50
2,N,Health Care,30.0,30
3,N,Self Care,20.0,20
4,N,Carro Maintenance,15.0,15
5,N,Ropa,15.0,15
6,W,Escuela,500.0,500
7,W,Isaac,20.0,20
8,W,Adri,30.0,30
9,W,Eliam,10.0,10


### Next Steps
- Plotly display
- Diplay in Streamlit
- Make it look nice
- Add tab to insert expenses

https://plotly.com/python/bullet-charts/
https://plotly.com/python/indicator/

In [ ]:
col_inter = ['Food', 'House Supplies', 'Self Care', 'Isaac', 'Adri', 'Eliam', 'Dates/Fun', 'Gifts', 'Fast Food', 'Junk Food']